# Traits

**Advantages** of traits

* Can be defined after the type is declared (unlike a supertype).
* Don't have to be created up-front, so types can be added later (unlike a Union).
* Otherwise-unrelated types (unlike a supertype) can be used.

Traits have a few **key parts**:

* Trait types: the different traits a type can have.
* Trait function: what traits a type has.
* Trait dispatch: using the traits.

To understand how traits work, it is important to understand the type of types in Julia. Types are values, so they have a type themselves: `DataType`. However, they also have the special pseudo-supertype `Type`, so a type `T` acts like `T<:Type{T}`.

In [ ]:
typeof(String) == DataType

In [ ]:
String isa Type{String}

In [ ]:
String isa Type{<:AbstractString}

### Trait type

This is the type that is used to attribute a particular trait.

In [ ]:
abstract type StatVariableType end

struct Continuous <: StatVariableType end
struct Categorical <: StatVariableType end
struct Ordinal <: StatVariableType end

### Trait function

The trait function takes a type as input, and returns an instance of the trait type. We use the trait function to declare what traits a particular type has. For example, we can say things like floats are continuous, booleans are categorical, etc.

In [ ]:
stattype(::Type{<:AbstractFloat}) = Continuous()
stattype(::Type{<:Bool}) = Categorical()
stattype(::Type{<:AbstractString}) = Categorical()
stattype(::Type{<:Integer}) = Ordinal()

### Using Traits

To use a trait we need to re-dispatch upon it. This is where we take the type of an input, and invoke the trait function on it to get objects of the trait type, then dispatch on those.

In [ ]:
# This is the trait re-dispatch
range_of_values(xs::AbstractVector{T}) where T = range_of_values(stattype(T), xs)

In [ ]:
using LinearAlgebra

# Dispatch on the trait
range_of_values(::Continuous, xs) = extrema(xs)
range_of_values(::Categorical, xs) = unique(xs)
range_of_values(::Ordinal, xs) = extrema(xs)

# range_of_values(::Union{Ordinal, Continuous}, xs) = extrema(xs)

In [ ]:
range_of_values([false, true, false, false, true])

In [ ]:
range_of_values([1.2, 4.8, 3.1, 11.9])

In [ ]:
@code_warntype debuginfo=:none range_of_values([1.2, 4.8, 3.1, 11.9])

In [ ]:
@code_native debuginfo=:none range_of_values([1.2, 4.8, 3.1, 11.9])

In [ ]:
@code_native debuginfo=:none extrema([1.2, 4.8, 3.1, 11.9])

## Extending and using Traits

### Indicate trait for new custom type

In [ ]:
abstract type Color end
struct Red <: Color end
struct Blue <: Color end
struct Green <: Color end

stattype(::Type{Color}) = Categorical()

In [ ]:
range_of_values([Red(), Blue(), Blue(), Green()])

In [ ]:
struct Measurement
    x::Float64
end

# make extrema work for our type
Base.isless(a::Measurement, b::Measurement) = a.x < b.x


stattype(::Type{Measurement}) = Continuous()

In [ ]:
ms = Measurement.(rand(3))

In [ ]:
range_of_values(ms)

### Extend current set of trait options

In [ ]:
# define new trait options
struct Normable <: StatVariableType end

# define trait function
stattype(::Type{<:Complex}) = Normable()

# use new traits
range_of_values(::Normable, xs) = maximum(norm.(xs))

In [ ]:
range_of_values([1+im,2im,3+2im,4,7+3im])

## Example from Base: `IteratorSize`

In [ ]:
typeof(1:10)

In [ ]:
supertypes(typeof(1:10))

In [ ]:
?Base.IteratorSize

In [ ]:
Base.IteratorSize(1:10)

In [ ]:
Base.IteratorSize(rand(2,3))

In [ ]:
@which Base.IteratorSize(rand(3))

Where is this trait utilized?

In [ ]:
BitArray(rand(Bool) for i in 1:3)

In [ ]:
@which BitArray(rand(Bool) for i in 1:3)

In [ ]:
Base.IteratorSize(rand(Bool) for i in 1:3)

In [ ]:
BitArray(Iterators.repeated(1))

In [ ]:
Base.IteratorSize(Iterators.repeated(1))

In [ ]:
@which BitArray(Iterators.repeated(1))

Resources:
* "Invented" by Tim Holy in this [github issue](https://github.com/JuliaLang/julia/issues/2345#issuecomment-54537633).
* See https://github.com/mauro3/Traits.jl#dispatch-on-traits for a detailed explanation and [SimpleTraits.jl](https://github.com/mauro3/SimpleTraits.jl) for a convenience implementation.
* Based on these blog posts:
 * https://invenia.github.io/blog/2019/11/06/julialang-features-part-2/
 * https://ahsmart.com/pub/holy-traits-design-patterns-and-best-practice-book.html